In [2]:
import overpy

In [3]:
api = overpy.Overpass()

In [5]:
api.url = 'https://overpass.kumi.systems/api/interpreter'

In [110]:
cnter = 0

In [105]:
class OSMArea:
    def __init__(self, name=None, idd=None, level=1, members=None):
        self.level = level
        self.id = idd
        self.members = members
        if members is None:
            self.members = list()
        self.name = name
    def __repr__(self):
        return 'OSMArea {} {}'.format(self.name, self.id) 

In [109]:
russia_area = OSMArea(idd=60189)

In [113]:
def deepen(rel_area: OSMArea, levels):
    global cnter
    cnter += 1
    if cnter % 100 == 0:
        print(cnter, levels, rel_area)
    result = api.query("""rel({}); out body;""".format(rel_area.id))
    rel = result.relations[0]
    rel_area.name = rel.tags['name']
    if levels == 0:
        return
    subareas = filter(lambda x: x.role == 'subarea' and 
                      type(x) == overpy.RelationRelation, rel.members)
    for relrel in subareas:
        rel_area.members.append(OSMArea(idd=relrel.ref))
    for relsub in rel_area.members:
        deepen(relsub, levels - 1)

In [114]:
deepen(russia_area, 3)

100 0 OSMArea None 1858728
200 0 OSMArea None 336201
300 0 OSMArea None 1396083
400 1 OSMArea None 115134
500 0 OSMArea None 1538158
600 0 OSMArea None 1746036
700 0 OSMArea None 398434
800 0 OSMArea None 1690647
900 2 OSMArea None 1221148
1000 0 OSMArea None 1074007
1100 0 OSMArea None 1149100
1200 0 OSMArea None 1100153
1300 0 OSMArea None 1399835
1400 0 OSMArea None 1749238
1500 0 OSMArea None 1114806
1600 0 OSMArea None 1029133
1700 0 OSMArea None 1319356
1800 0 OSMArea None 371580
1900 0 OSMArea None 3826848
2000 0 OSMArea None 610775
2100 0 OSMArea None 1046956
2200 0 OSMArea None 1278703
2300 0 OSMArea None 1102713
2400 0 OSMArea None 389663


In [135]:
import pickle

In [137]:
with open('osm_numbers.pickle', 'wb') as file:
    pickle.dump(russia_area, file)